In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd

In [3]:
import tensorflow

In [4]:
df_2018 = pd.read_csv('Resources/CONSOLIDADO_2018.csv', index_col=0)
df_2018.head()

,sem_x,codigo_x,dias_pag,dias_lab,sueldo,comp,asist,transp,vales_x,te_dob,...,t_per,t_ded,neto,antig,depto,puesto,label,tiempov,dist,delitos
0,1,7505,7.0,5.0,618.52,132.86,60.0,60.0,120.22,0.0,...,1029.86,21.63,1008.23,468,82,0003A,1,11.188,24.283333,0
1,1,6600,7.0,5.0,618.52,132.86,60.0,60.0,120.22,0.0,...,1029.86,22.60,1007.26,1187,80,0001,1,8.079,27.766667,46
2,1,7515,7.0,5.0,618.52,132.86,60.0,60.0,120.22,0.0,...,1029.86,21.53,1008.33,202,72,0001,0,8.400,21.900000,0
3,1,5933,7.0,5.0,618.52,132.86,60.0,60.0,120.22,0.0,...,1029.86,21.45,1008.41,1582,51,0001,1,3.738,10.183333,21
4,1,7170,7.0,5.0,618.52,132.86,60.0,60.0,120.22,0.0,...,1491.60,50.12,1441.48,774,57,0001,1,8.113,20.516667,0


In [5]:
df_2019 = pd.read_csv('Resources/CONSOLIDADO_2019.csv', index_col=0)
df_2019.head()

,sem_x,codigo_x,dias_pag,dias_lab,sueldo,comp,asist,transp,vales_x,te_dob,...,t_per,t_ded,neto,antig,depto,puesto,label,tiempov,dist,delitos
0,1,7840,7.0,5.0,1143.52,462.91,60.0,60.0,257.02,367.56,...,3351.59,340.18,3011.41,39,87,0185,0,4.940,6.433333,0
1,1,7505,7.0,5.0,721.00,173.04,60.0,60.0,143.04,0.00,...,1863.08,86.05,1777.03,520,82,0003A,0,11.188,24.283333,0
2,1,6600,7.0,5.0,721.00,30.38,60.0,50.0,120.22,180.25,...,1194.98,21.07,1173.91,1503,80,0002A,1,8.079,27.766667,46
3,1,7685,7.0,5.0,721.00,30.38,60.0,60.0,120.22,0.00,...,1056.69,28.04,1028.65,522,5,0002A,1,9.926,22.383333,0
4,1,7331,7.0,5.0,721.00,30.38,60.0,60.0,120.22,103.00,...,1129.56,24.71,1104.85,338,45,0001,0,8.901,17.116667,59


In [6]:
def labeling(row):
    sigue = "sigue"
    fue = "fue"
    if row == 0:
        return fue
    else:
        return sigue

In [7]:
df_2018["label"] = df_2018["label"].apply(lambda row: labeling(row))

In [8]:
df_2019["label"] = df_2019["label"].apply(lambda row: labeling(row))

In [9]:
#df_2019_Q1 = df_2019.loc[df_2019["sem_x"] <= 13]

In [10]:
df_2018["label"].value_counts()

sigue    7172
fue      2980
Name: label, dtype: int64

In [11]:
df_2019["label"].value_counts()

sigue    5552
fue      2661
Name: label, dtype: int64

In [12]:
#df_2019_Q1["label"].value_counts()

In [13]:
df_2018.isna().values.any()

False

In [14]:
df_2019.isna().values.any()

False

In [15]:
df_2018 = df_2018.set_index("codigo_x")

In [16]:
df_2019 = df_2019.set_index("codigo_x")

In [17]:
df_2018 = df_2018[["dias_pag","dias_lab","sueldo","comp","asist","transp","vales_x","te_dob",
          "te_trip","desc_Lab","fest_lab","dominic","perc_grav","obj","aguin","vac","prim_vac",
          "grat_esp","util","zapatos","otras","fonac","alim","infon","falt","enf","ries","antig",
          "tiempov","dist","delitos","label"]]

In [18]:
df_2019 = df_2019[["dias_pag","dias_lab","sueldo","comp","asist","transp","vales_x","te_dob",
          "te_trip","desc_Lab","fest_lab","dominic","perc_grav","obj","aguin","vac","prim_vac",
          "grat_esp","util","zapatos","otras","fonac","alim","infon","falt","enf","ries","antig",
          "tiempov","dist","delitos","label"]]

## Data Pre-Processing

In [19]:
X_train = df_2018.drop("label", axis=1)
X_test = df_2019.drop("label", axis=1)
y_train = df_2018["label"]
y_test = df_2019["label"]

In [20]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [21]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\dell\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [22]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [23]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [25]:
len(X_train_scaled[0])

31

In [26]:
len(X_test_scaled[0])

31

In [27]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=len(X_train_scaled[0])))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [28]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               3200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 13,502
Trainable params: 13,502
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 10152 samples
Epoch 1/100
10152/10152 - 1s - loss: 0.5502 - accuracy: 0.7260
Epoch 2/100
10152/10152 - 0s - loss: 0.4985 - accuracy: 0.7492
Epoch 3/100
10152/10152 - 0s - loss: 0.4621 - accuracy: 0.7768
Epoch 4/100
10152/10152 - 0s - loss: 0.4462 - accuracy: 0.7830
Epoch 5/100
10152/10152 - 0s - loss: 0.4243 - accuracy: 0.7942
Epoch 6/100
10152/10152 - 0s - loss: 0.4102 - accuracy: 0.8028
Epoch 7/100
10152/10152 - 0s - loss: 0.3978 - accuracy: 0.8110
Epoch 8/100
10152/10152 - 0s - loss: 0.3860 - accuracy: 0.8159
Epoch 9/100
10152/10152 - 0s - loss: 0.3743 - accuracy: 0.8248
Epoch 10/100
10152/10152 - 0s - loss: 0.3664 - accuracy: 0.8246
Epoch 11/100
10152/10152 - 0s - loss: 0.3556 - accuracy: 0.8329
Epoch 12/100
10152/10152 - 0s - loss: 0.3475 - accuracy: 0.8404
Epoch 13/100
10152/10152 - 0s - loss: 0.3403 - accuracy: 0.8345
Epoch 14/100
10152/10152 - 0s - loss: 0.3286 - accuracy: 0.8436
Epoch 15/100
10152/10152 - 0s - loss: 0.3256 - accuracy: 0.8428
Epoch 16/100
10152/10152 -

## Quantify our Trained Model

In [31]:
model_loss, model_accuracy = model.evaluate(
    X_train_scaled, y_train_categorical, verbose=2)
print(
    f"TRAINING - Loss: {model_loss}, Accuracy: {model_accuracy}")

10152/1 - 0s - loss: 0.0930 - accuracy: 0.9443
TRAINING - Loss: 0.134721790527715, Accuracy: 0.9443459510803223


In [32]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"TESTING - Loss: {model_loss}, Accuracy: {model_accuracy}")

8213/1 - 0s - loss: 2.0852 - accuracy: 0.7051
TESTING - Loss: 2.179290421868625, Accuracy: 0.7051016688346863


## Make Predictions

In [33]:
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [34]:
results = pd.DataFrame({
"prediction": prediction_labels,
"actual": y_test
})
results.head()

,prediction,actual
codigo_x,,
7840,sigue,fue
7505,sigue,fue
6600,sigue,sigue
7685,fue,sigue
7331,sigue,fue


In [35]:
results = results.reset_index()

In [36]:
results.to_csv("results.csv")

In [37]:
weights, biases = model.layers[1].get_weights()